In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [45]:
MTeams = pd.read_csv('MTeams.csv')
MSeasons = pd.read_csv('MSeasons.csv')
MNCAATourneySeeds = pd.read_csv('MNCAATourneySeeds.csv')
MRegularSeasonCompactResults = pd.read_csv('MRegularSeasonCompactResults.csv') 
MNCAATourneyCompactResults = pd.read_csv('MNCAATourneyCompactResults.csv')
SampleSubmissionStage1 = pd.read_csv('SampleSubmissionStage1.csv')
SampleSubmissionStage2 = pd.read_csv('SampleSubmissionStage2.csv')
MMasseyOrdinals = pd.read_csv('MMasseyOrdinals.csv')

In [4]:
MTeams.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2025
1,1102,Air Force,1985,2025
2,1103,Akron,1985,2025
3,1104,Alabama,1985,2025
4,1105,Alabama A&M,2000,2025


In [6]:
MSeasons.head()

,Season,DayZero,RegionW,RegionX,RegionY,RegionZ
0,1985,10/29/1984,East,West,Midwest,Southeast
1,1986,10/28/1985,East,Midwest,Southeast,West
2,1987,10/27/1986,East,Southeast,Midwest,West
3,1988,11/02/1987,East,Midwest,Southeast,West
4,1989,10/31/1988,East,West,Midwest,Southeast


In [8]:
MNCAATourneySeeds.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [24]:
MRegularSeasonCompactResults.tail()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
191791,2025,106,1461,69,1102,62,H,0
191792,2025,106,1462,76,1139,63,H,0
191793,2025,106,1466,80,1480,62,H,0
191794,2025,106,1468,94,1122,68,H,0
191795,2025,106,1474,89,1146,72,H,0


In [23]:
MNCAATourneyCompactResults.tail()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
2513,2024,146,1301,76,1181,64,N,0
2514,2024,146,1345,72,1397,66,N,0
2515,2024,152,1163,86,1104,72,N,0
2516,2024,152,1345,63,1301,50,N,0
2517,2024,154,1163,75,1345,60,N,0


In [18]:
SampleSubmissionStage1.head()

,ID,Pred
0,2021_1101_1102,0.5
1,2021_1101_1103,0.5
2,2021_1101_1104,0.5
3,2021_1101_1105,0.5
4,2021_1101_1106,0.5


In [22]:
SampleSubmissionStage2.head()

,ID,Pred
0,2025_1101_1102,0.5
1,2025_1101_1103,0.5
2,2025_1101_1104,0.5
3,2025_1101_1105,0.5
4,2025_1101_1106,0.5


In [73]:
SampleSubmissionStage1['Season'] = SampleSubmissionStage1['ID'].apply(lambda x: int(x.split('_')[0]))
SampleSubmissionStage1['Team1'] = SampleSubmissionStage1['ID'].apply(lambda x: int(x.split('_')[1]))
SampleSubmissionStage1['Team2'] = SampleSubmissionStage1['ID'].apply(lambda x: int(x.split('_')[2]))
SampleSubmissionStage1.head()

,ID,Pred,Season,Team1,Team2
0,2021_1101_1102,0.5,2021,1101,1102
1,2021_1101_1103,0.5,2021,1101,1103
2,2021_1101_1104,0.5,2021,1101,1104
3,2021_1101_1105,0.5,2021,1101,1105
4,2021_1101_1106,0.5,2021,1101,1106


In [26]:
MRegularSeason2021 = MRegularSeasonCompactResults[MRegularSeasonCompactResults['Season'] == 2021]
MRegularSeason2021.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
166880,2021,23,1101,70,1190,47,N,0
166881,2021,23,1104,81,1240,57,H,0
166882,2021,23,1111,81,1354,61,A,0
166883,2021,23,1113,94,1348,88,N,0
166884,2021,23,1114,71,1341,66,N,0


In [49]:
MMasseyOrdinals.head()

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
0,2003,35,SEL,1102,159
1,2003,35,SEL,1103,229
2,2003,35,SEL,1104,12
3,2003,35,SEL,1105,314
4,2003,35,SEL,1106,260


In [50]:
MMasseyOrdinals[MMasseyOrdinals['Season'] == 2021]['SystemName'].unique()

array(['AP', 'BNZ', 'BWE', 'DES', 'DII', 'DOK', 'EBP', 'HAS', 'INC',
       'JNG', 'LEF', 'MAS', 'MOR', 'PGH', 'POM', 'SAG', 'SMS', 'TRK',
       'TRP', 'USA', 'WIL', 'FAS', 'RTP', '7OT', 'BBT', 'CWL', 'DAV',
       'KPK', 'DCI', 'NOL', 'RT', 'SFX', 'SPR', 'SRS', 'TPR', 'DOL',
       'DUN', 'KCX', 'NET', 'COL', 'LOG', 'BIH', 'CBR', 'JJK', 'WLK',
       'KRA', 'SEL', 'WMV', 'WOB', 'ARG', 'PIR', 'EMK', 'FDM', 'LAW',
       'LMC', 'REW', 'RWP'], dtype=object)

In [51]:
MMasseyOrdinals[MMasseyOrdinals['Season'] == 2021]['RankingDayNum'].value_counts()

RankingDayNum
133    18076
114    17973
128    17759
121    17601
100    16908
107    16903
93     15856
86     14814
79     14463
72     12714
51     11302
65     11277
58     10886
44      8952
37      7167
30      6453
Name: count, dtype: int64

In [ ]:
from tqdm import tqdm

def record_record(Team1):
    Team1_wins = 0
    Team1_losses = 0
    for index, row in MRegularSeason2021.iterrows():
        if row['WTeamID'] == Team1:
            Team1_wins += 1
        if row['LTeamID'] == Team1:
            Team1_losses += 1
    return Team1_wins, Team1_losses

def avg_score_difference(Team1):
    Team1_score_difference = 0
    games = 0
    for index, row in MRegularSeason2021.iterrows():
        if row['WTeamID'] == Team1:
            Team1_score_difference += row['WScore'] - row['LScore']
            games += 1
        if row['LTeamID'] == Team1:
            Team1_score_difference += row['LScore'] - row['WScore']
            games += 1
    return Team1_score_difference/games if games != 0 else 0


from tqdm import tqdm
def weighted_avg_ranking(Team1, ranking_system, season):
    # Filter data once
    team_data = MMasseyOrdinals[(MMasseyOrdinals['Season'] == season) & 
                               (MMasseyOrdinals['TeamID'] == Team1) &
                               (MMasseyOrdinals['SystemName'] == ranking_system)]
    
    if team_data.empty:
        return 0  # Or another appropriate default value
    
    # Get unique ranking days
    unique_days = team_data['RankingDayNum'].unique()
    total_days = len(unique_days)
    
    # Initialize weights
    weights = range(total_days, 0, -1)
    
    # If there are fewer team_data rows than days, we need to adjust
    if len(team_data) < total_days:
        weights = weights[:len(team_data)]
    
    # Calculate weighted average
    weighted_sum = sum(row.OrdinalRank * weight for row, weight in tqdm(zip(team_data.itertuples(), weights), total=len(weights)))
    total_weight = sum(weights)
    
    return weighted_sum / total_weight if total_weight > 0 else 0

In [42]:
Model_Input_21 = MTeams[MTeams['FirstD1Season'] <= 2021]
Model_Input_21['Wins'] = Model_Input_21['TeamID'].apply(lambda x: record_record(x)[0])

C:\Users\nicko\AppData\Local\Temp\ipykernel_8692\3005028637.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Model_Input_21['Wins'] = Model_Input_21['TeamID'].apply(lambda x: record_record(x)[0])


In [43]:
Model_Input_21['avg_score_difference'] = Model_Input_21['TeamID'].apply(lambda x: avg_score_difference(x))

C:\Users\nicko\AppData\Local\Temp\ipykernel_8692\2008725154.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Model_Input_21['avg_score_difference'] = Model_Input_21['TeamID'].apply(lambda x: avg_score_difference(x))


In [68]:
Model_Input_21['POM_avg'] = Model_Input_21['TeamID'].apply(lambda x: weighted_avg_ranking(x, 'POM', 2021))

100%|██████████| 16/16 [00:00<?, ?it/s]


In [69]:
Model_Input_21

,TeamID,TeamName,FirstD1Season,LastD1Season,Wins,avg_score_difference,POM_avg
0,1101,Abilene Chr,2014,2025,19,14.565217,116.948529
1,1102,Air Force,1985,2025,5,-13.200000,277.375000
2,1103,Akron,1985,2025,13,3.809524,129.095588
3,1104,Alabama,1985,2025,24,9.800000,34.382353
4,1105,Alabama A&M,2000,2025,6,-3.866667,337.897059
...,...,...,...,...,...,...,...
366,1467,Merrimack,2020,2025,9,-0.388889,300.000000
367,1468,Bellarmine,2021,2025,11,4.277778,242.933824
368,1469,Utah Tech,2021,2025,6,-10.578947,319.625000
369,1470,Tarleton St,2021,2025,5,-4.266667,328.522059


In [86]:
SampleSubmissionStage1_2021 = SampleSubmissionStage1[SampleSubmissionStage1['Season'] == 2021]
SampleSubmissionStage1_2021.head()

# Merge for Team1
SampleSubmissionStage1_2021 = SampleSubmissionStage1_2021.merge(
    Model_Input_21[['TeamID', 'Wins', 'avg_score_difference', 'POM_avg']],
    left_on='Team1', right_on='TeamID', how='left'
).rename(columns={
    'Wins': 'Team1_wins',
    'avg_score_difference': 'Team1_avg_score_difference',
    'POM_avg': 'Team1_POM_avg'
}).drop(columns=['TeamID'])

# Merge for Team2
SampleSubmissionStage1_2021 = SampleSubmissionStage1_2021.merge(
    Model_Input_21[['TeamID', 'Wins', 'avg_score_difference', 'POM_avg']],
    left_on='Team2', right_on='TeamID', how='left'
).rename(columns={
    'Wins': 'Team2_wins',
    'avg_score_difference': 'Team2_avg_score_difference',
    'POM_avg': 'Team2_POM_avg'
}).drop(columns=['TeamID'])

SampleSubmissionStage1_2021.head()

,ID,Pred,Season,Team1,Team2,Team1_wins,Team1_avg_score_difference,Team1_POM_avg,Team2_wins,Team2_avg_score_difference,Team2_POM_avg
0,2021_1101_1102,0.5,2021,1101,1102,19.0,14.565217,116.948529,5.0,-13.200000,277.375000
1,2021_1101_1103,0.5,2021,1101,1103,19.0,14.565217,116.948529,13.0,3.809524,129.095588
2,2021_1101_1104,0.5,2021,1101,1104,19.0,14.565217,116.948529,24.0,9.800000,34.382353
3,2021_1101_1105,0.5,2021,1101,1105,19.0,14.565217,116.948529,6.0,-3.866667,337.897059
4,2021_1101_1106,0.5,2021,1101,1106,19.0,14.565217,116.948529,4.0,-7.333333,351.470588


In [87]:
# Merge for Team1
SampleSubmissionStage1_2021 = SampleSubmissionStage1_2021.merge(
    MRegularSeasonCompactResults[['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore']],
    left_on=['Season', 'Team1', 'Team2'],
    right_on=['Season', 'WTeamID', 'LTeamID'],
    how='left'
).rename(columns={
    'DayNum': 'Game1_DayNum',
    'WScore': 'Game1_WScore',
    'LScore': 'Game1_LScore',
    'WTeamID': 'Game1_WTeamID',
}).drop(columns=['LTeamID'])

# Merge for Team2
SampleSubmissionStage1_2021 = SampleSubmissionStage1_2021.merge(
    MRegularSeasonCompactResults[['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore']],
    left_on=['Season', 'Team2', 'Team1'],
    right_on=['Season', 'WTeamID', 'LTeamID'],
    how='left'
).rename(columns={
    'DayNum': 'Game2_DayNum',
    'WScore': 'Game2_WScore',
    'LScore': 'Game2_LScore',
    'WTeamID': 'Game2_WTeamID',
}).drop(columns=['LTeamID'])

SampleSubmissionStage1_2021.head()

,ID,Pred,Season,Team1,Team2,Team1_wins,Team1_avg_score_difference,Team1_POM_avg,Team2_wins,Team2_avg_score_difference,Team2_POM_avg,Game1_DayNum,Game1_WTeamID,Game1_WScore,Game1_LScore,Game2_DayNum,Game2_WTeamID,Game2_WScore,Game2_LScore
0,2021_1101_1102,0.5,2021,1101,1102,19.0,14.565217,116.948529,5.0,-13.200000,277.375000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021_1101_1103,0.5,2021,1101,1103,19.0,14.565217,116.948529,13.0,3.809524,129.095588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021_1101_1104,0.5,2021,1101,1104,19.0,14.565217,116.948529,24.0,9.800000,34.382353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021_1101_1105,0.5,2021,1101,1105,19.0,14.565217,116.948529,6.0,-3.866667,337.897059,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021_1101_1106,0.5,2021,1101,1106,19.0,14.565217,116.948529,4.0,-7.333333,351.470588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
train_21 = SampleSubmissionStage1_2021[SampleSubmissionStage1_2021['Game1_DayNum'].notnull()]
train_21.head()

,ID,Pred,Season,Team1,Team2,Team1_wins,Team1_avg_score_difference,Team1_POM_avg,Team2_wins,Team2_avg_score_difference,Team2_POM_avg,Game1_DayNum,Game1_WTeamID,Game1_WScore,Game1_LScore,Game2_DayNum,Game2_WTeamID,Game2_WScore,Game2_LScore
17,2021_1101_1122,0.5,2021,1101,1122,19.0,14.565217,116.948529,12.0,1.560000,177.691176,25.0,1101.0,80.0,72.0,NaN,NaN,NaN,NaN
37,2021_1101_1146,0.5,2021,1101,1146,19.0,14.565217,116.948529,4.0,-12.869565,265.095588,82.0,1101.0,93.0,58.0,117.0,1146.0,84.0,82.0
77,2021_1101_1190,0.5,2021,1101,1190,19.0,14.565217,116.948529,11.0,1.347826,160.941176,23.0,1101.0,70.0,47.0,NaN,NaN,NaN,NaN
108,2021_1101_1223,0.5,2021,1101,1223,19.0,14.565217,116.948529,5.0,-11.000000,343.764706,65.0,1101.0,66.0,63.0,NaN,NaN,NaN,NaN
109,2021_1101_1223,0.5,2021,1101,1223,19.0,14.565217,116.948529,5.0,-11.000000,343.764706,100.0,1101.0,88.0,59.0,NaN,NaN,NaN,NaN


In [ ]:
def classify(row):
    if not pd.isna(row['Game2_WTeamID']):
        return 1 if row['Game2_WTeamID'] == row['Team1'] else 0
    else:
        return 1 if row['Game1_WTeamID'] == row['Team1'] else 0

train_21['Classification'] = train_21.apply(classify, axis=1)
train_21.head()

In [90]:
import patsy as pt

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [98]:
from sklearn.metrics import brier_score_loss

# Define the features and target variable
features = ['Team1_wins', 'Team1_avg_score_difference', 'Team1_POM_avg', 
            'Team2_wins', 'Team2_avg_score_difference', 'Team2_POM_avg']
target = 'Classification'

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(train_21[features], train_21[target], test_size=0.2, random_state=42)

# Initialize the DecisionTreeClassifier
clf = RandomForestClassifier(random_state=40)

# Train the classifier
clf.fit(X_train, y_train)

# Predict probabilities
y_pred_proba = clf.predict_proba(X_test)[:, 1]

# Calculate the Brier score
brier_score = brier_score_loss(y_test, y_pred_proba)
print(f'Brier Score: {brier_score}')

Brier Score: 0.11636172839506173


In [99]:
# Predict the probabilities
y_pred_proba = clf.predict_proba(X_test)[:, 1]

# Predict the classifications
y_pred = clf.predict(X_test)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Display the predicted probabilities
predicted_probabilities = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred, 'Probability': y_pred_proba})
predicted_probabilities.head()

Accuracy: 0.8567901234567902


,Actual,Predicted,Probability
19901,1,1,0.78
29916,1,1,0.83
21515,1,1,0.91
17426,0,1,0.93
17203,1,1,1.00


In [102]:
import joblib

# Save the model to a file
joblib.dump(clf, 'random_forest_model.pkl')

['random_forest_model.pkl']

In [101]:
# Define the features for the entire 2021 sample set
features_2021 = SampleSubmissionStage1_2021[['Team1_wins', 'Team1_avg_score_difference', 'Team1_POM_avg', 
                                             'Team2_wins', 'Team2_avg_score_difference', 'Team2_POM_avg']]

# Predict probabilities for the entire 2021 sample set
SampleSubmissionStage1_2021['Pred'] = clf.predict_proba(features_2021)[:, 1]

# Use a 0.5 cutoff to create a 0/1 classification
SampleSubmissionStage1_2021['Classification'] = (SampleSubmissionStage1_2021['Pred'] > 0.5).astype(int)

# Display the updated dataframe with predictions
SampleSubmissionStage1_2021.head()

,ID,Pred,Season,Team1,Team2,Team1_wins,Team1_avg_score_difference,Team1_POM_avg,Team2_wins,Team2_avg_score_difference,Team2_POM_avg,Game1_DayNum,Game1_WTeamID,Game1_WScore,Game1_LScore,Game2_DayNum,Game2_WTeamID,Game2_WScore,Game2_LScore,Classification
0,2021_1101_1102,0.62,2021,1101,1102,19.0,14.565217,116.948529,5.0,-13.200000,277.375000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,2021_1101_1103,0.69,2021,1101,1103,19.0,14.565217,116.948529,13.0,3.809524,129.095588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,2021_1101_1104,0.53,2021,1101,1104,19.0,14.565217,116.948529,24.0,9.800000,34.382353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,2021_1101_1105,0.91,2021,1101,1105,19.0,14.565217,116.948529,6.0,-3.866667,337.897059,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,2021_1101_1106,0.93,2021,1101,1106,19.0,14.565217,116.948529,4.0,-7.333333,351.470588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [108]:
MNCAATourneyCompactResults.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [121]:
# Filter for the 2021 season
tourney_2021_results = MNCAATourneyCompactResults[MNCAATourneyCompactResults['Season'] == 2021][['WTeamID', 'LTeamID','DayNum']]
tourney_2021_results['Team1'] = np.where(tourney_2021_results['WTeamID'] < tourney_2021_results['LTeamID'], 
                                         tourney_2021_results['WTeamID'], 
                                         tourney_2021_results['LTeamID'])
tourney_2021_results['Team2'] = np.where(tourney_2021_results['WTeamID'] < tourney_2021_results['LTeamID'], 
                                         tourney_2021_results['LTeamID'], 
                                         tourney_2021_results['WTeamID'])
tourney_2021_results.drop(columns=['LTeamID'], inplace=True)
tourney_2021_results.head()

,WTeamID,DayNum,Team1,Team2
2251,1179,136,1179,1455
2252,1313,136,1111,1313
2253,1411,136,1291,1411
2254,1417,136,1277,1417
2255,1116,137,1116,1159


In [122]:
tourney_2021_results

,WTeamID,DayNum,Team1,Team2
2251,1179,136,1179,1455
2252,1313,136,1111,1313
2253,1411,136,1291,1411
2254,1417,136,1277,1417
2255,1116,137,1116,1159
...,...,...,...,...
2312,1211,148,1211,1425
2313,1417,148,1276,1417
2314,1124,152,1124,1222
2315,1211,152,1211,1417


In [129]:
# Merge the dataframes on Team1 and Team2
merged_results = SampleSubmissionStage1_2021.merge(
    tourney_2021_results,
    left_on=['Team1', 'Team2'],
    right_on=['Team1', 'Team2'],
    how='inner'
)

# Create the result column
merged_results['Result'] = merged_results.apply(lambda row: 1 if row['Team1'] == row['WTeamID'] else 0, axis=1)

# Select the required columns
final_results = merged_results[['Team1', 'Team2', 'WTeamID', 'Pred', 'Classification', 'Result', 'DayNum']]

# Order the results by DayNum
final_results = final_results.sort_values(by='DayNum').reset_index(drop=True)

final_results.head(50)

,Team1,Team2,WTeamID,Pred,Classification,Result,DayNum
0,1111,1313,1313,0.27,0,0,136
1,1291,1411,1411,0.17,0,0,136
2,1277,1417,1417,0.37,0,0,136
3,1179,1455,1179,0.80,1,1,136
4,1210,1260,1260,0.44,0,0,137
5,1361,1393,1393,0.74,1,0,137
6,1333,1397,1333,0.14,0,1,137
7,1326,1331,1331,0.91,1,0,137
8,1317,1345,1317,0.61,1,1,137
9,1314,1458,1458,0.58,1,0,137


In [130]:
from sklearn.metrics import accuracy_score, brier_score_loss

# Extract the actual results and predicted probabilities
y_true = final_results['Result']
y_pred_proba = final_results['Pred']
y_pred = final_results['Classification']

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy}')

# Calculate Brier score
brier_score = brier_score_loss(y_true, y_pred_proba)
print(f'Brier Score: {brier_score}')

Accuracy: 0.6515151515151515
Brier Score: 0.23773333333333332
